# Part 5. 값 정제(Cleaning)의 대표 패턴
이 장은 본격적인 데이터 전처리. 데이터의 "값 자체"를 분석 가능한 상태로 만드는 단계다. <br>
dtype와 결측치 처리가 끝났다면, 이제 남은 문제는 값의 표기와 일관성이다.

이장의 목표는 간단하다.
> 같은 의미의 값은 항상 같은 형태로 존재하게 만든다.

In [1]:
import pandas as pd

df = pd.DataFrame({
    " Date ": ["2026-01-01", "2026-01-01", "2026-01-02", None,        "2026-01-02"],
    " Menu ": [" Latte ",   " latte",      "Americano ", "Mocha",     "Americano "],
    " Price ": ["5,000원",   "5,000원",     None,         "5,500원",   None],
    " Qty ":   [1,           1,            2,            None,        2],
    "   Paid": ["TRUE", "false", True, None, "1"],
    " memo ":  ["test",      "test",       "dup",        "x",         "dup"]
})

print("원본 DataFrame")
print(df)

print("\nDataFrame 정보")
print(df.info())

원본 DataFrame
        Date        Menu   Price   Qty    Paid memo 
0  2026-01-01      Latte   5,000원   1.0   TRUE  test
1  2026-01-01       latte  5,000원   1.0  false  test
2  2026-01-02  Americano     None   2.0   True   dup
3        None       Mocha  5,500원   NaN   None     x
4  2026-01-02  Americano     None   2.0      1   dup

DataFrame 정보
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0    Date    4 non-null      object 
 1    Menu    5 non-null      object 
 2    Price   3 non-null      object 
 3    Qty     4 non-null      float64
 4      Paid  4 non-null      object 
 5    memo    5 non-null      object 
dtypes: float64(1), object(5)
memory usage: 368.0+ bytes
None


## 1) 컬럼 구조 정리 (rename / drop)
값을 보기 전에 먼저 컬럼 구조부터 안정화해야 한다.

컬럼 정리의 목적
- 컬럼 이름이 한눈에 이해되도록
- 불필요한 컬럼을 제거해서 혼란을 줄이기
- 이후 코드 가독성을 높이기

이 단계에서는 주로 다음 작업을 한다.
- 컬럼명 공백 제거
- 대소문자 통일
- 의미가 모호한 이름 변경
- 분석에 필요 없는 컬럼 제거

In [2]:
print("=== 컬럼명 표준화 전 ===")
print(df.columns)

# 컬럼명 공백 제거 + 소문자 통일: " Date " -> "date"
df.columns = df.columns.str.strip().str.lower()
# str 문자열 접근자
# strip() 값의 공백 제거(양옆)
# lower() 소문자로 변환

print("\n=== 컬럼명 표준화 후 ===")
print(df.columns)

=== 컬럼명 표준화 전 ===
Index([' Date ', ' Menu ', ' Price ', ' Qty ', '   Paid', ' memo '], dtype='object')

=== 컬럼명 표준화 후 ===
Index(['date', 'menu', 'price', 'qty', 'paid', 'memo'], dtype='object')


In [3]:
# 컬럼명 바꾸기: 의미가 더 명확한 이름으로
df = df.rename(columns={"menu": "menu_name"})
print(df.columns)

Index(['date', 'menu_name', 'price', 'qty', 'paid', 'memo'], dtype='object')


In [4]:
# 필요 없는 컬럼 제거: 분석에 안 쓰는 메모 컬럼 같은 것
df = df.drop(columns=["memo"])

print("=== 컬럼 정리 후 df ===")
print(df)
print("\n=== df.info() ===")
df.info()

=== 컬럼 정리 후 df ===
         date   menu_name   price  qty   paid
0  2026-01-01      Latte   5,000원  1.0   TRUE
1  2026-01-01       latte  5,000원  1.0  false
2  2026-01-02  Americano     None  2.0   True
3        None       Mocha  5,500원  NaN   None
4  2026-01-02  Americano     None  2.0      1

=== df.info() ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       4 non-null      object 
 1   menu_name  5 non-null      object 
 2   price      3 non-null      object 
 3   qty        4 non-null      float64
 4   paid       4 non-null      object 
dtypes: float64(1), object(4)
memory usage: 328.0+ bytes


## 2) 문자열 정제 기본 4종 (공백 / 대소문자 / 불필요 문자 / 값 치환)
문자열 문제의 본질은 하나다. 같은 의미인데 표기가 다르다.

문자열 정제는 대부분 아래 4개의 문제들 중 하나다.
1. 공백 문제
2. 대소문자 문제
3. 불필요 문자
4. 값 치환 (다음에)

문자열 문제의 핵심은 "표기 흔들림"을 없애는 것이다.<br>
같은 값인데 다르게 저장되어 있으면 그룹핑/필터링/조인이 전부 깨진다.

### 공백 문제
### 대소문자 문제

In [5]:
print("=== menu_name 정제 ===")
df["menu_clean"] = (
    df["menu_name"]
    .astype("string")
    .str.strip() # 공백제거
    .str.lower() # 소문자 통일
)

df[["menu_name", "menu_clean"]]

=== menu_name 정제 ===


,menu_name,menu_clean
0,Latte,latte
1,latte,latte
2,Americano,americano
3,Mocha,mocha
4,Americano,americano


### 공백 문제
### 불필요 문자

In [6]:
print("\n=== price 문자열 정제 ===")
df["price_clean"] = (
    df["price"]
    .astype("string")                   # 숫자형을 문자열로 강제 변환
    .str.strip() # 공백제거
    .str.replace(",", "", regex=False) # 치환, 쉼표를 없앰
    .str.replace("원", "", regex=False) # 치환, 원을 없앰
    
)

df[["price", "price_clean"]]


=== price 문자열 정제 ===


,price,price_clean
0,"5,000원",5000
1,"5,000원",5000
2,None,<NA>
3,"5,500원",5500
4,None,<NA>


## 3) 문자열 → 숫자 변환 표준 패턴 (to_numeric)
숫자 계산이 목적이라면 문자열 상태는 반드시 끝내야 한다.<br>
문자열 → 숫자 변환은 항상 같은 순서로 진행한다.
1. 문자열로 통일
2. 불필요한 문자 제거 (문자열 정제로 여기까지 수행)
3. 숫자로 파싱 (to_numeric)
4. 실패 값은 결측으로 표시

In [7]:
print("=== price 숫자 파싱 ===")
df["price_clean"] = pd.to_numeric(df["price_clean"], errors="coerce")
print(df[["price", "price_clean"]])

print("\n=== price 결측치 개수 ===")
print(df["price_clean"].isna().sum())

=== price 숫자 파싱 ===
    price  price_clean
0  5,000원         5000
1  5,000원         5000
2    None         <NA>
3  5,500원         5500
4    None         <NA>

=== price 결측치 개수 ===
2


In [8]:
print("\n=== qty 숫자 파싱(안전 변환) ===")
df["qty_clean"] = pd.to_numeric(df["qty"], errors="coerce")

print(df[["qty", "qty_clean"]])
print("\n=== qty 결측치 개수 ===")
print(df["qty_clean"].isna().sum())


=== qty 숫자 파싱(안전 변환) ===
   qty  qty_clean
0  1.0        1.0
1  1.0        1.0
2  2.0        2.0
3  NaN        NaN
4  2.0        2.0

=== qty 결측치 개수 ===
1


### 결측 처리 전략 선택 (연습용: 0으로 대체)
연습에서는 0으로 채웠지만 실무에서는 "0이 의미 있는 값인가?"를 반드시 질문해야 한다.

In [9]:
print("\n=== 결측치 대체(연습용) ===")
df_fill = df.copy()
df_fill["price_clean"] = df_fill["price_clean"].fillna(0)
df_fill["qty_clean"]   = df_fill["qty_clean"].fillna(0)

df_fill[["price", "price_clean", "qty", "qty_clean"]]


=== 결측치 대체(연습용) ===


,price,price_clean,qty,qty_clean
0,"5,000원",5000,1.0,1.0
1,"5,000원",5000,1.0,1.0
2,None,0,2.0,2.0
3,"5,500원",5500,NaN,0.0
4,None,0,2.0,2.0


## 4) 범주값 통일 패턴 (TRUE / True / false 문제)
범주형 데이터는 표기가 다르면 완전히 다른 값으로 인식된다. <br>
이 문제를 방치하게 되면
- 그룹화 결과가 쪼개진다.
- 집계 값이 왜곡된다.

범주값 통일의 기본 흐름은 다음과 같다.
1. 문자열 기준으로 통일한다
2. 공백과 대소문자를 정리한다
3. 의미 기준으로 하나의 값으로 매핑한다
4. 매핑되지 않은 값은 따로 확인한다

### 문자열로 통일
### 공백/대소문자 차이를 제거

In [10]:
print("=== paid 문자열 통일(정리 전) ===")
print(df_fill["paid"])

paid_norm = (
    df_fill["paid"]
    .astype("string")   # 문자열 기준으로 통일
    .str.strip()        # 앞뒤 공백 제거
    .str.upper()        # 대소문자 통일
)

print("\n=== paid_norm (정리 후) ===")
print(paid_norm)

=== paid 문자열 통일(정리 전) ===
0     TRUE
1    false
2     True
3     None
4        1
Name: paid, dtype: object

=== paid_norm (정리 후) ===
0     TRUE
1    FALSE
2     TRUE
3     <NA>
4        1
Name: paid, dtype: string


### 의미 기준으로 하나의 값으로 매핑
아주 잠깐 짚고 넘어가기



map은 Series의 각 값을 미리 정의한 "값 대응표"에 따라 치환하는 함수다. <br>
즉, "이 값이면 → 저 값으로" 라는 규칙을 적용한다.

문장구조
```python
Series.map(기존 값 → 새 값의 대응표)
```

In [11]:
paid_map = {
    "TRUE": True,  "FALSE": False,
    "Y": True,     "N": False,
    "YES": True,   "NO": False,
    "1": True,     "0": False,
    "T": True,     "F": False,
}

df_fill["paid_bool"] = paid_norm.map(paid_map)

print("\n=== paid_bool 변환 결과 ===")
print(df_fill[["paid", "paid_bool"]])



=== paid_bool 변환 결과 ===
    paid paid_bool
0   TRUE      True
1  false     False
2   True      True
3   None       NaN
4      1      True


이때 매핑에 실패한 값은 따로 살펴본다.

In [12]:
unknown = df_fill[df_fill["paid_bool"].isna()][["paid"]]
unknown

,paid
3,None


## 5) 중복 데이터의 판단과 처리 (duplicated / drop_duplicates) !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
중복 데이터는 값이 같은지보다 의미가 같은지가 중요하다. <br>
중복을 다룰 때는 항상 두 가지를 먼저 정해야 한다.
- 어떤 컬럼 기준으로 같은 데이터인가?
- 첫 번째를 남길지, 마지막을 남길지?

중복 처리에서 기억할 것은 두 가지 옵션뿐이다.
- subset: 중복 판단 기준
- keep: first(첫 행 유지) / last(마지막 행 유지)

지금의 데이터를 예를 들어<br>
같은 날짜 + 같은 메뉴 + 같은 수량 + 같은 가격 + 같은 결제여부면 같은 주문이라고 가정한다.

In [13]:
df_fill

,date,menu_name,price,qty,paid,menu_clean,price_clean,qty_clean,paid_bool
0,2026-01-01,Latte,"5,000원",1.0,TRUE,latte,5000,1.0,True
1,2026-01-01,latte,"5,000원",1.0,false,latte,5000,1.0,False
2,2026-01-02,Americano,None,2.0,True,americano,0,2.0,True
3,None,Mocha,"5,500원",NaN,None,mocha,5500,0.0,NaN
4,2026-01-02,Americano,None,2.0,1,americano,0,2.0,True


In [14]:
df_dup = df_fill[["date", "menu_clean", "qty_clean", "price_clean", "paid_bool"]]
key_cols = ["date", "menu_clean", "qty_clean", "price_clean", "paid_bool"]

In [15]:
print("=== 중복 여부 마스크 ===")
dup_mask = df_dup.duplicated(subset=key_cols, keep="first")
print(dup_mask)

=== 중복 여부 마스크 ===
0    False
1    False
2    False
3    False
4     True
dtype: bool


In [16]:
print("\n=== 중복 제거 전 ===")
print(df_dup)

print("\n=== 중복 제거(첫 번째 유지) ===")
dedup_first = df_dup.drop_duplicates(subset=key_cols, keep="first")
print(dedup_first)

print("\n=== 중복 제거(마지막 유지) ===")
dedup_last = df_dup.drop_duplicates(subset=key_cols, keep="last")
print(dedup_last)


=== 중복 제거 전 ===
         date menu_clean  qty_clean  price_clean paid_bool
0  2026-01-01      latte        1.0         5000      True
1  2026-01-01      latte        1.0         5000     False
2  2026-01-02  americano        2.0            0      True
3        None      mocha        0.0         5500       NaN
4  2026-01-02  americano        2.0            0      True

=== 중복 제거(첫 번째 유지) ===
         date menu_clean  qty_clean  price_clean paid_bool
0  2026-01-01      latte        1.0         5000      True
1  2026-01-01      latte        1.0         5000     False
2  2026-01-02  americano        2.0            0      True
3        None      mocha        0.0         5500       NaN

=== 중복 제거(마지막 유지) ===
         date menu_clean  qty_clean  price_clean paid_bool
0  2026-01-01      latte        1.0         5000      True
1  2026-01-01      latte        1.0         5000     False
3        None      mocha        0.0         5500       NaN
4  2026-01-02  americano        2.0            0   